## LSTM RNN

In [536]:
import warnings
import numpy as np
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import yfinance as yf
import matplotlib.pyplot as plt

In [537]:
warnings.filterwarnings('ignore')

In [538]:
aark = pd.DataFrame(yf.download('ARKK', '2020-03-01','2022-03-31')['Adj Close'].apply(lambda x: round(x,2)))

[*********************100%***********************]  1 of 1 completed


In [539]:
display(aark.head(),aark.tail())

,Adj Close
Date,
2020-03-02,52.79
2020-03-03,51.73
2020-03-04,53.49
2020-03-05,52.24
2020-03-06,50.87


,Adj Close
Date,
2022-03-24,66.40
2022-03-25,64.51
2022-03-28,67.06
2022-03-29,71.39
2022-03-30,68.68


In [540]:
aark.hvplot()

:Curve   [Date]   (Adj Close)

In [541]:
training_data_len = len(aark[:-60])

In [542]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(aark.values)

In [543]:
train_data, test_data = scaled_data[0:training_data_len, :], scaled_data[training_data_len - 60:, :]

X_train,X_test, y_train, y_test = [],[],[],aark.values[training_data_len:, :]

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i,0])
    
for i in range(60, len(test_data)):
    X_test.append(test_data[i-60:i, 0])

In [544]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1 ))

In [545]:
model = Sequential()

In [546]:
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [547]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_44 (LSTM)              (None, 60, 50)            10400     
                                                                 
 lstm_45 (LSTM)              (None, 50)                20200     
                                                                 
 dense_44 (Dense)            (None, 25)                1275      
                                                                 
 dense_45 (Dense)            (None, 1)                 26        
                                                                 
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________


In [548]:
model.compile(optimizer='adam', loss='mse')

In [549]:
model.fit(X_train, y_train, batch_size=1, epochs=1)

406/406 [==============================] - 6s 10ms/step - loss: 0.0077


In [550]:
predictions = model.predict(X_test) 

2/2 [==============================] - 0s 7ms/step


In [552]:
train = aark[:training_data_len]
valid = aark[training_data_len:]
valid['Predictions'] = scaler.inverse_transform(predictions)
train.rename(columns={'Adj Close':'Training Data'}, inplace=True)
valid.rename(columns={'Adj Close':'Actual Adjusted Close', 'Predictions': 'Forecasted Adjusted Close'}, inplace=True)
train['Training Data'].hvplot() * valid[['Actual Adjusted Close', 'Forecasted Adjusted Close']].hvplot()

:Overlay
   .Curve.Training_Data :Curve   [Date]   (Training Data)
   .NdOverlay.I         :NdOverlay   [Variable]
      :Curve   [Date]   (value)

In [553]:
display(valid.head(),valid.tail())

,Actual Adjusted Close,Forecasted Adjusted Close
Date,,
2022-01-04,92.69,104.543617
2022-01-05,86.12,104.374489
2022-01-06,85.58,103.604210
2022-01-07,84.42,102.494690
2022-01-10,84.64,101.155746


,Actual Adjusted Close,Forecasted Adjusted Close
Date,,
2022-03-24,66.40,69.751190
2022-03-25,64.51,70.657043
2022-03-28,67.06,71.401245
2022-03-29,71.39,72.160027
2022-03-30,68.68,73.153694


In [554]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

69.8010157216986